# Projet : BaristaBot — Assistant de Commande de Café avec LangGraph + Gemini API


Ce projet a pour but de créer un agent conversationnel capable de :
- Prendre des commandes de boissons (café/thé) en langage naturel
- Gérer dynamiquement un menu avec un outil
- Confirmer ou modifier les commandes
- Utiliser une structure d'état persistante avec LangGraph
- Intégrer l'API Gemini via LangChain


In [ ]:
!pip install -qU langgraph==0.2.45 langchain-google-genai==2.0.4

In [ ]:

import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [ ]:

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class OrderState(TypedDict):
    messages: Annotated[list, add_messages]
    order: list[str]
    finished: bool

BARISTABOT_SYSINT = (
    "system",
    "Tu es BaristaBot, un assistant de commande pour un café. Tu guides le client dans sa commande "
    "à partir d’un menu. Tu ne parles que des produits du menu, pas de sujets hors sujet. "
    "Tu dois vérifier les articles, utiliser les fonctions disponibles (add_to_order, confirm_order, place_order, etc.), "
    "et confirmer la commande avant de la soumettre."
)

WELCOME_MSG = "Bienvenue chez BaristaBot ! Tapez `q` pour quitter. Que puis-je vous servir aujourd’hui ?"


In [ ]:

from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

def chatbot(state: OrderState) -> OrderState:
    messages = [BARISTABOT_SYSINT] + state["messages"]
    return {"messages": [llm.invoke(messages)]}


In [ ]:

from pprint import pprint

user_msg = ("user", "Je voudrais un cappuccino s’il vous plaît.")
state = {"messages": [user_msg], "order": [], "finished": False}
state = chatbot(state)

for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")


In [ ]:

from langchain_core.messages.ai import AIMessage

def human_node(state: OrderState) -> OrderState:
    print("Bot:", state["messages"][-1].content)
    user_input = input("Vous : ")

    if user_input.lower() in {"q", "quit"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}

def chatbot_with_welcome_msg(state: OrderState) -> OrderState:
    if state["messages"]:
        output = llm.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        output = AIMessage(content=WELCOME_MSG)
    return state | {"messages": [output]}


In [ ]:

from typing import Literal

def maybe_exit_human_node(state: OrderState) -> Literal["chatbot", "__end__"]:
    return END if state.get("finished") else "chatbot"


In [ ]:

from langchain_core.tools import tool

@tool
def get_menu() -> str:
    return '''
    MENU:
    - Espresso, Americano, Cold Brew
    - Latte, Cappuccino, Cortado, Mocha, Flat White
    - Chai Latte, Matcha Latte, Earl Grey, London Fog
    - Modificateurs : lait (entier, avoine, amande), chaud/froid, décaféiné, etc.
    - Soy milk est en rupture de stock.
    '''


In [ ]:

from langgraph.prebuilt import ToolNode

tools = [get_menu]
tool_node = ToolNode(tools)
llm_with_tools = llm.bind_tools(tools)

def maybe_route_to_tools(state: OrderState) -> str:
    msg = state["messages"][-1]
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        return "tools"
    return "human"

def chatbot_with_tools(state: OrderState) -> OrderState:
    if state["messages"]:
        output = llm_with_tools.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        output = AIMessage(content=WELCOME_MSG)
    return state | {"messages": [output]}


In [ ]:

from collections.abc import Iterable
from langchain_core.messages.tool import ToolMessage
from random import randint

@tool
def add_to_order(drink: str, modifiers: Iterable[str]) -> str: pass

@tool
def confirm_order() -> str: pass

@tool
def get_order() -> str: pass

@tool
def clear_order(): pass

@tool
def place_order() -> int: pass

def order_node(state: OrderState) -> OrderState:
    tool_msg = state["messages"][-1]
    order = state.get("order", [])
    outbound_msgs = []
    order_placed = False

    for call in tool_msg.tool_calls:
        name, args = call["name"], call["args"]
        if name == "add_to_order":
            drink = args["drink"]
            mods = ", ".join(args["modifiers"]) or "sans modificateur"
            order.append(f"{drink} ({mods})")
            response = "
".join(order)
        elif name == "confirm_order":
            print("Commande :")
            for item in order:
                print(" -", item)
            response = input("Est-ce correct ? ")
        elif name == "get_order":
            response = "
".join(order) or "(aucune commande)"
        elif name == "clear_order":
            order.clear()
            response = "Commande vidée"
        elif name == "place_order":
            print("Commande envoyée à la cuisine !")
            order_placed = True
            response = f"Temps estimé : {randint(1, 5)} minutes"
        else:
            raise NotImplementedError(name)

        outbound_msgs.append(ToolMessage(content=response, name=name, tool_call_id=call["id"]))

    return {"messages": outbound_msgs, "order": order, "finished": order_placed}


In [ ]:

graph_builder = StateGraph(OrderState)

graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("ordering", order_node)

graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
graph_builder.add_conditional_edges("human", maybe_exit_human_node)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("ordering", "chatbot")
graph_builder.add_edge(START, "chatbot")

graph_with_order_tools = graph_builder.compile()


In [ ]:

config = {"recursion_limit": 100}
state = graph_with_order_tools.invoke({"messages": [], "order": [], "finished": False}, config)
